> GBM, LGBM 모델에 따라 적절한 Parameter setting 값이  다를 수 있음

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np

In [ ]:
!git clone https://github.com/yun-seon/DX-Training-LGES-0630.git

Cloning into 'DX-Training-LGES-0630'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 102 (delta 12), reused 0 (delta 0), pack-reused 68
Receiving objects: 100% (102/102), 14.48 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
# import dataset

data = pd.read_csv('/content/DX-Training-LGES-0630/Data/Data_wafer_sensor_classification.csv')

del data['Unnamed: 0']
data.head()

,0,1,2,3,4,5,6,7,8,9,...,711,712,713,714,715,716,717,718,719,y
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-97.0,9.436896e-16,0.0,-1.000000e+00,6.505213e-18,1.517883e-18,-2.710505e-20,0.000000e+00,0.000000e+00,0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-96.0,-9.600000e+01,-6.0,6.938894e-17,5.551115e-17,-1.000000e+00,-1.734723e-18,-2.059984e-18,1.084202e-19,0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-97.0,-3.774758e-15,-1.0,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,5.551115e-17,-1.000000e+00,0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,-116.0,-1.160000e+02,-116.0,-1.160000e+02,-1.160000e+02,-2.000000e+00,-1.000000e+00,1.387779e-17,-3.469447e-18,0
4,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-115.0,-1.150000e+02,-115.0,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,-1.150000e+02,0


In [ ]:
from sklearn.model_selection import train_test_split

# split train and test data (test ratio: 30%)
X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, 0:720], data.iloc[:,720], test_size = 0.3)

In [ ]:
# 성능 비교를 위한 Boosting 모델 정의
lgb = LGBMClassifier()
gbm = GradientBoostingClassifier()

lreg = LogisticRegression()

In [ ]:
# 최적의 파라미터 값 찾기 위한 범위 설정
  
# LGB                        
param_lgb = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             "max_depth" : [25, 50, 75],
             "num_leaves" : [100,300,500,900,1200],
             "n_estimators" : [100, 200, 300,500,800,1000],
             "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5]
              }
# GBM              
param_gbm = {"max_depth" : [4,5,6,7,8,9,10],
             "learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
             "n_estimators" : [100,200,300,500]
              }

# Logistic
param_lreg = { 'C' : [1.0, 3, 5, 7, 10],
              'max_iter': [50, 200, 100, 300, 500,700, 800]
              }

In [ ]:
# 최적의 파라미터 찾기 위한 GridsearchCV 실행
# cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)
gscv_lgb = GridSearchCV (estimator = lgb, param_grid = param_lgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_gbm = GridSearchCV (estimator = gbm, param_grid = param_gbm, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_lreg = GridSearchCV (estimator = lreg, param_grid = param_lreg, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_lgb.fit(X_train, Y_train)
gscv_gbm.fit(X_train, Y_train)
gscv_lreg.fit(X_train, Y_train)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time=   2.1s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time=   1.3s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=100; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time=   1.5s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time=   1.3s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=300; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=500; total time=   1.6s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=500; total time=   1.3s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=500; total time=   1.1s
[CV] END learning_rate=0.01, max_depth=25, n_estimators=100, num_leaves=

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

[CV] END .................................C=1.0, max_iter=50; total time=   0.1s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................C=1.0, max_iter=200; total time=   0.2s


...

[CV] END .................................C=10, max_iter=500; total time=   0.6s
[CV] END .................................C=10, max_iter=700; total time=   0.6s
[CV] END .................................C=10, max_iter=700; total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END .................................C=10, max_iter=700; total time=   0.8s
[CV] END .................................C=10, max_iter=800; total time=   0.6s
[CV] END .................................C=10, max_iter=800; total time=   0.8s
[CV] END .................................C=10, max_iter=800; total time=   0.7s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=1,
             param_grid={'C': [1.0, 3, 5, 7, 10],
                         'max_iter': [50, 200, 100, 300, 500, 700, 800]},
             scoring='accuracy', verbose=2)

In [ ]:
print("="*30)
print('LGB 파라미터: ', gscv_lgb.best_params_)
print('LGB 예측 정확도: {:.4f}'.format(gscv_lgb.best_score_))

print("="*30)
print('GBM 파라미터: ', gscv_gbm.best_params_)
print('GBM 예측 정확도: {:.4f}'.format(gscv_gbm.best_score_))

print("="*30)
print('Lreg 파라미터: ', gscv_lreg.best_params_)
print('Lreg 예측 정확도: {:.4f}'.format(gscv_lreg.best_score_))
print("="*30)

LGB 파라미터:  {'learning_rate': 0.4, 'max_depth': 25, 'n_estimators': 100, 'num_leaves': 100}
LGB 예측 정확도: 0.9843
GBM 파라미터:  {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 500}
GBM 예측 정확도: 0.9867
Lreg 파라미터:  {'C': 1.0, 'max_iter': 300}
Lreg 예측 정확도: 0.9843
